In [1]:
import pandas as pd
pd.set_option('display.max_columns',1000)

In [2]:
tele = pd.read_excel('./2018.xlsx')
df1 = pd.read_excel('./17.11-18.5.xlsx')
df2 = pd.read_excel('./18.6-18.11.xlsx')

In [4]:
tele = tele.dropna(how='all')
tele = tele.drop_duplicates(subset=['工单号'],keep='first')
#len(tele)#非空有22714条，1人重复，共有22713条有效记录
len(tele)

22713

In [3]:
df = df1.append(df2)#共有26667条有效记录


In [43]:
tele[tele['IMEI']=='RN120180511083545772']#1人工单号重复

,IMEI,姓名,联系电话,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
8256,RN120180511083545772,刘良跃,18352529796,Berkeley-TL20,A0000082E01CDD,鲁蔚,继续维修,NaT,NaN,2018-05-11 08:35:00
8258,RN120180511083545772,刘良跃,18352529796,Berkeley-TL20,A0000082E01CDD,鲁蔚,继续维修,2018-05-11 08:54:00,徐建,2018-05-11 08:35:00


In [71]:
tele[tele['工单号'].str.contains(' ')]#有四人工单号缺失

,工单号,姓名,联系电话,Unnamed: 3,IMEI,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
17634,,李绍鹏,18951586998,London-AL30B,A000008E18D2E9,叶晓燕,继续维修,2018-09-21 18:02:00,周君,2018-09-21 17:39:00
18226,,钱栋,13771490027,Rhone-AL00,A000007FC89FCF,左平,继续维修,2018-09-29 18:03:00,徐正明,2018-09-29 17:45:00
21578,,钱哲锋,13921150545,HZ-W19,BC7574E198BF,许嘉莉,继续维修,2018-11-19 14:07:00,顾麒麟,2018-11-19 11:09:00
22581,,李骏,18921161061,NEO-AL00D,A000008CBBC97A,鲁蔚,继续维修,2018-12-07 13:19:00,徐建,2018-12-04 14:26:00


In [5]:
df = pd.merge(df,tele[['工单号','姓名','联系电话']],on='工单号',how='left')

In [113]:
#df['受理开始时间'] = df['受理开始时间'].astype('datetime64[ns]')
df['受理开始时间']=pd.to_datetime(df['受理开始时间'],format='%Y-%m-%d %H:%M:%S')

In [127]:
#早7晚9
hourarr = [i.hour for i in df["受理开始时间"]]
for i in hourarr:
    if i<=7 or i>=21:
        print (i)

In [130]:
#psid不符
sn_imei = df.drop_duplicates(subset=('IMEI','SN'),keep='first')

TypeError: 'method' object is not subscriptable

In [136]:
dup_imei = sn_imei.groupby(['SN'])['IMEI'].size().reset_index()
dup_imei=dup_imei[dup_imei['IMEI']>=2]

In [141]:
dup_imei_df=pd.merge(dup_imei[['SN']],df,on='SN',how='left')#359条psid不符
dup_imei_df[['SN','IMEI']]

In [227]:
dup_imei_df.to_excel('PSID不符.xlsx',index=False)

In [145]:
#SN不符
dup_sn = sn_imei.groupby(['IMEI'])['SN'].size().reset_index()
dup_sn = dup_sn[dup_sn['SN']>=2]
dup_sn_df=pd.merge(dup_sn[['IMEI']],df,on='IMEI',how='left')
#dup_sn_df[['SN','IMEI']]

In [155]:
imei_tele = df.drop_duplicates(subset=('IMEI','联系电话'),keep='first')

In [157]:
#一号多修
dup_imei_1tele = imei_tele.groupby(['联系电话'])['IMEI'].size().reset_index()
dup_imei_1tele = dup_imei_1tele[dup_imei_1tele['IMEI']>=2]
dup_imei_1tele_df=pd.merge(dup_imei_1tele[['联系电话']],df,on='联系电话',how='left')

In [228]:
dup_imei_1tele_df.to_excel('一号多修.xlsx',index=False)

In [159]:
#一机多号
dup_tele_1imei = imei_tele.groupby(['IMEI'])['联系电话'].size().reset_index()
dup_tele_1imei = dup_tele_1imei[dup_tele_1imei['联系电话']>=2]
dup_tele_1imei_df=pd.merge(dup_tele_1imei[['IMEI']],df,on='IMEI',how='left')

In [229]:
dup_tele_1imei_df.to_excel('一机多号.xlsx',index=False)

In [161]:
#购机日期不符
imei_buy = df.drop_duplicates(subset=('IMEI','购买日期'),keep='first')
dup_buy = imei_buy.groupby(['IMEI'])['购买日期'].size().reset_index()
dup_buy = dup_buy[dup_buy['购买日期']>=2]
dup_buy_df=pd.merge(dup_buy[['IMEI']],df,on='IMEI',how='left')#包含可疑

In [230]:
dup_buy_df.to_excel('购机日期不符（包含可疑）.xlsx',index=False)

In [168]:
temp = dup_buy_df[['IMEI','购买日期']].dropna().drop_duplicates(subset=('IMEI','购买日期'),keep='first')

In [172]:
doubt_dup_buy=temp.drop_duplicates(subset=('IMEI'),keep=False)
have_dup_buy=temp.drop_duplicates(subset=('IMEI'),keep='first').append(doubt_dup_buy).drop_duplicates(subset=('IMEI'),keep=False)

In [175]:
have_dup_buy_df = pd.merge(have_dup_buy[['IMEI']],df,on='IMEI',how='left')#不包含可疑

In [231]:
have_dup_buy_df.to_excel('购机日期不符.xlsx',index=False)

In [182]:
#产品型号不符
imei_type = df.drop_duplicates(subset=('IMEI','产品型号'),keep='first')
dup_type = imei_type.groupby(['IMEI'])['产品型号'].size().reset_index()
dup_type = dup_type[dup_type['产品型号']>=2]
dup_type_df=pd.merge(dup_type[['IMEI']],df,on='IMEI',how='left')

In [232]:
dup_type_df.to_excel('产品型号不符.xlsx',index=False)

In [209]:
#重复出现
new_imei = df.dropna(subset=['新IMEI','IMEI'])


In [210]:
new_imei_copy = new_imei.copy()
new_imei_copy = new_imei_copy.rename(columns={'新IMEI':'新新IMEI'})
new_imei_copy = new_imei_copy.rename(columns={'IMEI':'新IMEI'})

In [211]:
new_new_imei = pd.merge(new_imei,new_imei_copy[['新IMEI','新新IMEI']],on='新IMEI',how='inner')

In [218]:
repeat = new_new_imei.drop(['新IMEI'],axis=1).rename(columns={'新新IMEI':'新IMEI'})#93个人换过两次

repeat1 = new_imei.append(repeat)

repeat1[repeat1.duplicated(subset=['IMEI','新IMEI'],keep=False)]

In [224]:
new_imei_copy = new_imei.copy()
new_imei_copy = new_imei_copy.rename(columns={'新IMEI':'新新新IMEI'})
new_imei_copy = new_imei_copy.rename(columns={'IMEI':'新新IMEI'})
new_new_new_imei = pd.merge(new_new_imei,new_imei_copy[['新新IMEI','新新新IMEI']],on='新新IMEI',how='inner')
len(new_new_new_imei)

4

In [225]:
repeat0 = new_new_new_imei.drop(['新IMEI','新新IMEI'],axis=1).rename(columns={'新新新IMEI':'新IMEI'})#93个人换过两次

repeat2 = new_imei.append(repeat0)

repeat2[repeat2.duplicated(subset=['IMEI','新IMEI'],keep=False)]

,Bmt,IMEI,SN,SR单号,SR满意度,SR状态,Spdt,二次判保状态,产品型号,产品域,产品描述,产品族,产品系列,产品线,代表处,使用天数,使用时长,使用替代关系标记,供应商,供应商编码,保修凭证,保修凭证号码,出借时间,创建时间,取号时间,取机完成时间,取机方式,取机满意度,取机责任人,取机责任人编码,受理员,受理员编码,受理完成时间,受理开始时间,受理渠道,品牌,回寄单发货时间,回寄单签收时间,国家/地区,地区部,城市,备件消耗,备用机IMEI,备用机SN号,备用机产品描述,备用机状态,姓名,寄送单创建时间,寄送单发货时间,寄送单签收时间,工单保外收入,工单修改标示,工单号,待料开始时间,待料物料描述,待料物料编码,待料结束时间,总金额（正常）,所有维修方法,押金金额,换出物料,收费类型,放弃维修原因（RN）,放弃维修原因（SR）,故障大类1,故障大类1编码,故障大类2,故障大类2编码,故障大类3,故障大类3编码,故障小类1,故障小类1编码,故障小类2,故障小类2编码,故障小类3,故障小类3编码,故障描述,故障机类型,数量,新IMEI,新SN,是否已收费,是否待料,是否曾经待料,是否曾经留机,最终判保状态,最终收费金额,最终确认维修方案,权益名称,来源,检测完工时间,检测工程师,检测工程师编码,检测结果,沟通完成时间,沟通方式,沟通确认结果,沟通负责人,沟通负责人编码,物料费,用户是否留机,省份,确认沟通结果的备注,礼品名称,类型,系统判保状态,维修完工时间,维修工程师,维修工程师编码,维修方案,维修方法,维修费（正常）,联系电话,购买日期,软件版本,返还时间,门店名称,门店类型,门店编码,集成结果,集成说明,非本BOM物料标记,首次判保状态


In [203]:
df[df['新IMEI']=='A00000775777AC'][['IMEI','新IMEI']]

,IMEI,新IMEI


In [19]:
#一机多料
consume = df.dropna(subset=['备件消耗'])

In [14]:
df[['备件消耗']].isnull().sum()

备件消耗    6995
dtype: int64

In [21]:
consume=consume[(consume['维修方案']!='人为损转保修')&(consume['维修方案']!='保外维修')]

In [25]:
for index, row in consume.iterrows():
    if '&(' in row['备件消耗']:
        print (row['备件消耗'])

(02351KPW)前壳服务组件-Chopin-AL00A-TP/LCD&前壳组件-黑色-ODM&(03032RSE)主板(服务备板)-Chopin-AL00A-1.4GHz-EMCP 32GB-LPDDR3 3GB-中国-渠道
(04070857)信号电缆-USB数据线,白色-1m-USB A公-OD3.2-Micro B公-终端专用&(97060AFL)USB小板-Chopin-AL00A-QL2020A_KB_V4_PCBA
(02220682)适配器--10degC-45degC-100V-240V-5V/9V 2A-中式 2Pin/USB2.0-智能快充-CoC V5 Tier2/DoE VI-新ID-白色-HUAWEI Logo-终端专用&(04071121)信号电缆-5V3A USB2.0-1m-Type C-OD 3.5-Usb A-终端专用
(02351KBC)天线小板组件-Barca-AL00A-服务组件-比亚迪&(04071121)信号电缆-5V3A USB2.0-1m-Type C-OD 3.5-Usb A-终端专用
(02351KJK)前壳服务组件-BAC-AL00-黑色-前壳/LCD&TP/听筒装饰件-比亚迪&(03032RBF)主板(服务备板)-Barca-TL00A
(02351KGT)前壳服务组件-Barca-AL00A-白色-通达-前壳/LCD&TP/听筒装饰件&(22020269)扬声器模组-7ohm-100-20kHz-1W-39.2*20.36*3.55mm-Barca-终端专用
(02351KPV)前壳服务组件-Chopin-AL00A-TP/LCD&前壳组件-白色-ODM&(97060ABN)USB小板-Chopin-W09B-QL2020F_KB_V4_PCBA&(97060ABT)LINK-KB FPC-Chopin-W09B-QL2020A_LINK-KB_FPC组件
(02351KGT)前壳服务组件-Barca-AL00A-白色-通达-前壳/LCD&TP/听筒装饰件&(03032RBF)主板(服务备板)-Barca-TL00A
(02351LFB)后壳组件-Chopin-W09B-皎月白-中文通用镭雕-B壳备件-内置锂电池*1(B类)-服务组件&(03032RNQ)主板(服务备板)-Chopin-W09B-1.4

(02350YRX)前壳服务组件-前壳+LCD&TP+受话器+电池-Berlin-AL10A-银色-旺鑫&(03024BMT)Berlin USB副板-USB小板(硬板)-TL00_TL10-2*4
(02350YRX)前壳服务组件-前壳+LCD&TP+受话器+电池-Berlin-AL10A-银色-旺鑫&(02351PJS)小板服务组件-Berlin-AL10A-USB副板
(02351JMK)小板组件-Prague-AL00A-USB副板&(04071002)信号电缆-USB2.0-5V/1A-1.0m-USB A公-MicroB 公-OD 3.2mm-白色-低烟无卤-终端专用
(02351DEW)带电池的前壳服务组件-Prague-AL00C-流光金/铝合金+塑胶/宏讯/荣耀-前壳/LCD&TP/受话器/电池&(02351JMK)小板组件-Prague-AL00A-USB副板&(03032KKV)主板(服务备板)-Prague-AL00A
(02351JMK)小板组件-Prague-AL00A-USB副板&(02351MPA)带电池的前壳服务组件-Prague-AL00C-幻海蓝/铝合金+塑胶/比亚迪/荣耀-前壳/LCD&TP/受话器/电池
(02351JMK)小板组件-Prague-AL00A-USB副板&(03032KKV)主板(服务备板)-Prague-AL00A
(03024BMT)Berlin USB副板-USB小板(硬板)-TL00_TL10-2*4&(03032NWD)主板(服务备板)-Berlin-TL10-移动新规范定制版
(02350YRX)前壳服务组件-前壳+LCD&TP+受话器+电池-Berlin-AL10A-银色-旺鑫&(03024BMT)Berlin USB副板-USB小板(硬板)-TL00_TL10-2*4
(02351FPJ)天线小板组件-DUK-AL20,手机副板维修备板-服务组件-长盈&(03032MPU)主板(服务备板)-Duke-AL20
(02350YFR)电池盖指纹服务组件-Berlin-AL10C-服务组件-金色-富士康杭州&(02350YRX)前壳服务组件-前壳+LCD&TP+受话器+电池-Berlin-AL10A-银色-旺鑫&(02351PJS)小板服务组件-Berlin-AL10A-USB副板&(2202024

In [32]:
dup_wuliao = consume[consume['备件消耗'].str.contains("&\(")]

In [35]:
dup_wuliao.to_excel('一机多料.xlsx',index=False)